<a href="https://colab.research.google.com/github/viniciusrpb/datavis_book/blob/main/cap5_intro_ml_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar as bibliotecas do Pandas, Numpy, Sklearn

In [81]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

#from sklearn.preprocessing import MinMaxScaler

**Leitura do conjunto de dados**

In [43]:
rawData = pd.read_csv("sample_data/students2.csv")

Atributos e seus valores no conjunto de dados:

Performance {'Excellent','Vg','Good','Average'}
Gender {'male','female'}
Caste {'General','OBC','SC','ST'}
coaching {'NO','WA','OA'}
time {'ONE','TWO','THREE','FOUR','FIVE','SEVEN'}
Class_ten_education {'SEBA','OTHERS','CBSE'}
twelve_education {'AHSEC','CBSE','OTHERS'}
medium' {'ENGLISH','OTHERS','ASSAMESE'}
Class_ X_Percentage {'Excellent','Vg','Good','Average'}
Class_XII_Percentage {'Excellent','Vg','Good','Average'}
Father_occupation {'DOCTOR','SCHOOL_TEACHER','BUSINESS','COLLEGE_TEACHER','OTHERS','BANK_OFFICIAL','ENGINEER','CULTIVATOR'}
Mother_occupation {'OTHERS','HOUSE_WIFE','SCHOOL_TEACHER','DOCTOR','COLLEGE_TEACHER','BANK_OFFICIAL','BUSINESS','CULTIVATOR','ENGINEER'}

In [44]:
print(rawData.head())

  Performance Gender  ... Father_occupation Mother_occupation
0   Excellent   male  ...            DOCTOR            OTHERS
1   Excellent   male  ...    SCHOOL_TEACHER        HOUSE_WIFE
2   Excellent   male  ...          BUSINESS        HOUSE_WIFE
3   Excellent   male  ...    SCHOOL_TEACHER    SCHOOL_TEACHER
4   Excellent   male  ...   COLLEGE_TEACHER        HOUSE_WIFE

[5 rows x 12 columns]


Verificar a quantidade de instâncias e atributos

In [45]:
print(rawData.shape)

(666, 12)


Remover instâncias com valores ausentes

In [46]:
rawData = rawData.replace('?',np.NaN)
rawData = rawData.dropna(axis=0)

Ver os atributos do conjunto de dados

In [47]:
attributes = rawData.columns

Ver os valores do atributo 'Performance'

In [48]:
print(rawData['Performance'].value_counts())

Good         210
Vg           198
Average      157
Excellent    101
Name: Performance, dtype: int64


Reservar o atributo de label

In [49]:
label = rawData['Performance']



Verificar os valores e suas frequências no atributo 'time'

In [50]:
print(rawData['time'].value_counts())

TWO      368
ONE      199
THREE     86
FOUR      11
FIVE       1
SEVEN      1
Name: time, dtype: int64


Converte atributos categórico ordinal para numérico (inteiro)

---



In [51]:
#converte atributo time de objeto para categórico
rawData['time'] = pd.Categorical(rawData['time'])

rawData['time_num'] = rawData['time'].cat.codes

In [52]:
print(rawData.head())

rawData = rawData.drop(['time','Performance'],axis=1)

  Performance Gender    Caste  ... Father_occupation Mother_occupation time_num
0   Excellent   male  General  ...            DOCTOR            OTHERS        2
1   Excellent   male      OBC  ...    SCHOOL_TEACHER        HOUSE_WIFE        5
2   Excellent   male      OBC  ...          BUSINESS        HOUSE_WIFE        5
3   Excellent   male  General  ...    SCHOOL_TEACHER    SCHOOL_TEACHER        2
4   Excellent   male  General  ...   COLLEGE_TEACHER        HOUSE_WIFE        5

[5 rows x 13 columns]


In [53]:
print(rawData.head())

  Gender    Caste coaching  ... Father_occupation Mother_occupation time_num
0   male  General       NO  ...            DOCTOR            OTHERS        2
1   male      OBC       WA  ...    SCHOOL_TEACHER        HOUSE_WIFE        5
2   male      OBC       OA  ...          BUSINESS        HOUSE_WIFE        5
3   male  General       WA  ...    SCHOOL_TEACHER    SCHOOL_TEACHER        2
4   male  General       OA  ...   COLLEGE_TEACHER        HOUSE_WIFE        5

[5 rows x 11 columns]


Conversão de um atributo nominal para atributo binário (dummy variables)

In [54]:
rawData = pd.get_dummies(rawData)

print(rawData.head())

   time_num  ...  Mother_occupation_SCHOOL_TEACHER
0         2  ...                                 0
1         5  ...                                 0
2         5  ...                                 0
3         2  ...                                 1
4         5  ...                                 0

[5 rows x 44 columns]


In [55]:
rawData

,time_num,Gender_female,Gender_male,Caste_General,Caste_OBC,Caste_SC,Caste_ST,coaching_NO,coaching_OA,coaching_WA,Class_ten_education_CBSE,Class_ten_education_OTHERS,Class_ten_education_SEBA,twelve_education_AHSEC,twelve_education_CBSE,twelve_education_OTHERS,medium_ASSAMESE,medium_ENGLISH,medium_OTHERS,Class_ X_Percentage_Average,Class_ X_Percentage_Excellent,Class_ X_Percentage_Good,Class_ X_Percentage_Vg,Class_ XII_Percentage_Average,Class_ XII_Percentage_Excellent,Class_ XII_Percentage_Good,Class_ XII_Percentage_Vg,Father_occupation_BANK_OFFICIAL,Father_occupation_BUSINESS,Father_occupation_COLLEGE_TEACHER,Father_occupation_CULTIVATOR,Father_occupation_DOCTOR,Father_occupation_ENGINEER,Father_occupation_OTHERS,Father_occupation_SCHOOL_TEACHER,Mother_occupation_BANK_OFFICIAL,Mother_occupation_BUSINESS,Mother_occupation_COLLEGE_TEACHER,Mother_occupation_CULTIVATOR,Mother_occupation_DOCTOR,Mother_occupation_ENGINEER,Mother_occupation_HOUSE_WIFE,Mother_occupation_OTHERS,Mother_occupation_SCHOOL_TEACHER
0,2,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,5,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,5,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,2,0,1,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,5,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,2,1,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
662,4,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
663,5,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
664,4,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [56]:
X = rawData.values

print(X)

[[2 0 1 ... 0 1 0]
 [5 0 1 ... 1 0 0]
 [5 0 1 ... 1 0 0]
 ...
 [5 0 1 ... 0 0 1]
 [4 0 1 ... 1 0 0]
 [2 1 0 ... 0 1 0]]


In [57]:
attributes_dummies = rawData.columns

normalize = preprocessing.MinMaxScaler()
xscaled = normalize.fit_transform(X)

dados_tratados = pd.DataFrame(xscaled,columns=attributes_dummies)

dados_tratados

,time_num,Gender_female,Gender_male,Caste_General,Caste_OBC,Caste_SC,Caste_ST,coaching_NO,coaching_OA,coaching_WA,Class_ten_education_CBSE,Class_ten_education_OTHERS,Class_ten_education_SEBA,twelve_education_AHSEC,twelve_education_CBSE,twelve_education_OTHERS,medium_ASSAMESE,medium_ENGLISH,medium_OTHERS,Class_ X_Percentage_Average,Class_ X_Percentage_Excellent,Class_ X_Percentage_Good,Class_ X_Percentage_Vg,Class_ XII_Percentage_Average,Class_ XII_Percentage_Excellent,Class_ XII_Percentage_Good,Class_ XII_Percentage_Vg,Father_occupation_BANK_OFFICIAL,Father_occupation_BUSINESS,Father_occupation_COLLEGE_TEACHER,Father_occupation_CULTIVATOR,Father_occupation_DOCTOR,Father_occupation_ENGINEER,Father_occupation_OTHERS,Father_occupation_SCHOOL_TEACHER,Mother_occupation_BANK_OFFICIAL,Mother_occupation_BUSINESS,Mother_occupation_COLLEGE_TEACHER,Mother_occupation_CULTIVATOR,Mother_occupation_DOCTOR,Mother_occupation_ENGINEER,Mother_occupation_HOUSE_WIFE,Mother_occupation_OTHERS,Mother_occupation_SCHOOL_TEACHER
0,0.4,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,0.4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
662,0.8,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
663,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
664,0.8,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


0      Excellent
1      Excellent
2      Excellent
3      Excellent
4      Excellent
         ...    
661      Average
662      Average
663      Average
664      Average
665      Average
Name: Performance, Length: 666, dtype: object


Estratégia de avaliação dos classificadores: Holdout Cross Validation

In [62]:
X = dados_tratados.values;
y = label.values;

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

Classificador SVM: criação do classificador e treinamento

In [78]:
clfsvm = SVC(kernel="rbf",gamma='scale',C=1)

clfsvm.fit(X_train,y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Realiza os testes

In [79]:
y_classificados = clfsvm.predict(X_test)

Análise da performance do classificador

In [80]:
print(classification_report(y_test,y_classificados))

              precision    recall  f1-score   support

     Average       0.77      0.77      0.77        61
   Excellent       1.00      0.02      0.04        51
        Good       0.39      0.63      0.48        75
          Vg       0.42      0.44      0.43        80

    accuracy                           0.49       267
   macro avg       0.64      0.46      0.43       267
weighted avg       0.60      0.49      0.45       267



In [88]:
clfmlp = MLPClassifier(random_state=1, max_iter=300,hidden_layer_sizes=(10,10))

clfmlp.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [89]:
y_classif_mlp = clfmlp.predict(X_test)

In [90]:
print(classification_report(y_test,y_classif_mlp))

              precision    recall  f1-score   support

     Average       0.78      0.82      0.80        61
   Excellent       0.67      0.08      0.14        51
        Good       0.39      0.56      0.46        75
          Vg       0.44      0.49      0.46        80

    accuracy                           0.51       267
   macro avg       0.57      0.49      0.47       267
weighted avg       0.55      0.51      0.48       267

